# Module 1 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
#       hills      5
~       swamp      7
x       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
....*..
...***.
.###...
..##...
..#..**
....***
.......
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [30]:
full_world = [
  ['.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '#', 'x', 'x', '#', '#'], 
  ['.', '.', '.', '.', '#', 'x', 'x', 'x', '*', '*', '*', '*', '~', '~', '*', '*', '*', '*', '*', '.', '.', '#', '#', 'x', 'x', '#', '.'], 
  ['.', '.', '.', '#', '#', 'x', 'x', '*', '*', '.', '.', '~', '~', '~', '~', '*', '*', '*', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.'], 
  ['.', '#', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '~', '~', '~', '~', '~', '.', '.', '.', '.', '.', '#', 'x', '#', '.', '.'], 
  ['.', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '.', '.', '.', '#', '.', '.', '.'], 
  ['.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '#', '#', '#', '.', '.'], 
  ['.', '.', '.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '.', '~', '~', '.', '.', '#', '#', '#', '.', '.', '.'], 
  ['.', '.', '.', '~', '~', '~', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '~', '.', '#', '#', '#', '.', '.', '.', '.'], 
  ['.', '.', '~', '~', '~', '~', '~', '.', '#', '#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.', '.', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['~', '~', '~', '~', '~', '.', '.', '#', '#', 'x', 'x', '#', '.', '~', '~', '~', '~', '.', '.', '.', '#', 'x', '#', '.', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '.', '.', '#', '*', '*', '#', '.', '.', '.', '.', '~', '~', '~', '~', '.', '.', '#', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', '.', '.', '*', '*', '*', '*', '#', '#', '#', '#', '.', '~', '~', '~', '.', '.', '#', 'x', '#', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '.', '~', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '.', '.', 'x', 'x', 'x', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~'], 
  ['.', '.', '#', '#', '#', '#', 'x', 'x', '*', '*', '*', '*', '*', '.', 'x', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', '*', '*', 'x', 'x', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '#', '#', '.', '.', '~', '~', '~', '~', '~', '~'], 
  ['.', '#', '#', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '#', '#', '.', '~', '~', '~', '~', '~'], 
  ['#', 'x', '#', '#', '#', '#', '.', '.', '.', '.', '.', 'x', 'x', 'x', '#', '#', 'x', 'x', '.', 'x', 'x', '#', '#', '~', '~', '~', '~'], 
  ['#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', 'x', 'x', '#', '#', '#', '#', 'x', 'x', 'x', '~', '~', '~', '~'], 
  ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '#', '#', '#', '.', '.', '.']]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [31]:
test_world = [
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [6]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [7]:
COSTS = { '.': 1, '*': 3, '#': 5, '~': 7}

## A\* Search Implementation

As Python is an interpreted language, you're going to need to insert all of your helper functions *before* the actual `a_star_search` function implementation.

Please **read the Blackboard** for information about the expected code structure. I expect a "literate" style of "functional" programming (nothing fancy: only define functions, no class definitions, pass state between functions, isolate side effects like printing). Notebooks do not lend themselves to OOP very well.

-----

In [5]:
# add as many markdown and code cells here as you need for helper functions.

### Helper Functions

The helper function, size_of_world, accepts the state representation of the world as a list of lists and returns its maximum x-coordinate and its maximum y-coordinate in Cartesian space as integers.

In [5]:

def size_of_world( world ):
    maxYcoord = len( world ) - 1
    maxXcoord = len( world[0] ) - 1
    return( maxXcoord, maxYcoord )

# Exercise code
maxXcoord, maxYcoord = size_of_world( test_world )
print("maxXcoord of test world: ", maxXcoord)
print("maxYcoord of test world: ", maxYcoord)


maxXcoord of test world:  6
maxYcoord of test world:  6


The helper function, sum_tuples, accepts an action (or move) as a tuple as well as the coordinates that describe the current state as a tuple. The coordinates are contained in the form (x-coordinate, y_coordinate). The x-coordinates are summed and the y-coordinates are summed. The return type is a tuple and it corresponds to the next state to which an agent would move.

In [8]:

def sum_tuples( this_move, this_state ):
    
    xNextState = this_state[0] + this_move[0]
    yNextState = this_state[1] + this_move[1]
    
    next_state = (xNextState, yNextState)
    
    return( next_state )

# Exercise code
a_current_state = ( 5, 4 )
print( "Beginning at the following state: ", a_current_state )
an_action = ( 1, 0 )
print( "and moving one position to the right" )
resulting_state = sum_tuples( an_action, a_current_state )
print( "results in this new location: ", resulting_state )

Beginning at the following state:  (5, 4)
and moving one position to the right
results in this new location:  (6, 4)


This function, calculate_next_states, passes in the current state argument as a tuple in the form (x-coordinate, y-coordinate); a list containing tuples of the form (x-coordinate, y-coordinate) corresponding to actions/moves that can be made by an agent; and a state representation of the world in the form of a list of lists. The function calculates the coordinates of the next states that are possible based upon the actions represented in the moves list. A list containing tuples that correspond to those next states is returned. 

In [9]:

def calculate_next_states( state, moves, world ):

    maxXcoord, maxYcoord = size_of_world( world )
    next_states = []
    
    # move up
    if state[1] != 0:
        move = moves[0]
        upState = sum_tuples( move, state )
    else:
        upState = state
    next_states.append( upState )
    
    # move right
    if state[0] < maxXcoord:
        move = moves[1]
        rightState = sum_tuples( move, state )
    else:
        rightState = state
    next_states.append( rightState )
    
    # move down
    if state[1] < maxYcoord:
        move = moves[2]
        downState = sum_tuples( move, state )
    else:
        downState = state
    next_states.append( downState )
    
    # move left
    if state[0] != 0:
        move = moves[3]
        leftState = sum_tuples( move, state )
    else:
        leftState = state
    next_states.append( leftState )
        
    return( next_states )

# Exercise code
this_state = ( 4, 3 )
print( "If an agent is currently in the following state within the test world: ", this_state )
list_of_moves = [(0, -1), (1, 0), (0, 1), (-1, 0)]
print( "and has the following actions available: ", list_of_moves )
these_next_states = calculate_next_states( this_state, list_of_moves, test_world )
print( "The next states would be at the following locations: ", these_next_states )

If an agent is currently in the following state within the test world:  (4, 3)
and has the following actions available:  [(0, -1), (1, 0), (0, 1), (-1, 0)]
The next states would be at the following locations:  [(4, 2), (5, 3), (4, 4), (3, 3)]


This function, find_successors, returns the eligible child nodes of the current state. A dictionary containing information about the current state of an agent is passed in as stateDict. The function also accepts a list containing tuples that correspond to the actions available to an agent. A state representation of the world in the form of a list of lists is also passed in. The function ensures that an action results in a new state (i.e. that an action does not cause the agent to move outside the coordinates of the world). In addition, the cost of each move is calculated as are g(n), h(n), and f(n). The function returns a list of children in the form of a tuple containing another tuple with (f(n), location of child state) and a dictionary containing the values of g(n), h(n), and the previous node visited.


In [21]:

def find_successors( stateDict, moves, world ):
    
    path_from_previous_node = stateDict["path"]
    g_n_from_previous_node = stateDict["g"]
    
    # initialize empty list to store the children
    children = []
    # determine the new coordinates of the states resulting from the available actions
    potential_child_nodes = calculate_next_states( stateDict["state"], moves, world )
    
    # check whether the new states are the same as the current state i.e. if the moves were possible.
    # If equal to current state, the state was on an edge of the map
    # and the move could not be performed without falling off the map
        
    for child_index in range( 0, len(potential_child_nodes) ):
        # check to make sure there is indeed a new state that was moved to
        if potential_child_nodes[child_index] != stateDict["state"]:
            
            if potential_child_nodes[child_index] in stateDict["path"]:
                continue
            
            # determine symbol for that position
            xCoord = potential_child_nodes[child_index][0]
            yCoord = potential_child_nodes[child_index][1]
            
            # if action results in a forbidden state, continue
            if world[yCoord][xCoord] == "x":
                continue
            
            # determine cost of that action
            cost_of_action = COSTS[world[yCoord][xCoord]]
            
            g_node = g_n_from_previous_node + cost_of_action
            h_node = my_heuristic( stateDict["state"], goal )
            f_node = g_node + h_node
            
            # assemble information into a dictionary
            this_dict = {}
            this_dict["state"] = potential_child_nodes[child_index]
            this_dict["h"] = h_node
            this_dict["g"] = g_node
                        
            path_so_far = []
            for j in range( 0, len(path_from_previous_node) ):
                path_so_far.append( path_from_previous_node[j] )
            # append coordinate for this state to the path
            path_so_far.append( potential_child_nodes[child_index] )            
            this_dict["path"] = path_so_far
           
            # attach f(n) to the dictionary
            key_as_tuple = ( f_node, potential_child_nodes[child_index] )
            this_tuple = ( key_as_tuple, this_dict )
            #print("this tuple: ", this_tuple)
            children.append( this_tuple )
    
    return( children )


### Funcition: my_heuristic 

The function, my_heuristic, accepts the current state in the form of a tuple (x-coordinate, y_coordinate) and the location of the goal state as a tuple (x-coordinate, y_coordinate). The Manhattan distance between the two is calculated and returned as an integer. Formally, this value is h(n), which is the estimated distance between the current state and the goal state.


In [18]:
# change the formal arguments and the return value to be what you need.
def my_heuristic( currentState, goalState ):
    # extract x-coordinate of the current state
    x_coord_current_state = currentState[0]
    # extract x-coordinate of the goal state
    x_coord_goal_state = goalState[0]
    
    # extract y-coordinate of the current state
    y_coord_current_state = currentState[1]
    # extract y-coordinate of the goal state
    y_coord_goal_state = goalState[1]
    
    # calculate distance in the x direction
    x_distance = abs( x_coord_current_state - x_coord_goal_state )
    # calculate distance in the y direction
    y_distance = abs( y_coord_current_state - y_coord_goal_state )
    
    h_distance = x_distance + y_distance
    
    return( h_distance )

# Exercise code
current = ( 3, 2 )
print( "With a current state of: ", current )
goal = ( 6, 6 )
print( "and a goal state of: ", goal )
this_distance = my_heuristic( current, goal )
print( "The Manhattan distance is: ", this_distance )

With a current state of:  (3, 2)
and a goal state of:  (6, 6)
The Manhattan distance is:  7


**a_star_search**

The `a_star_search` function uses the A\* Search algorithm to solve a navigational problem for an agent in a grid world. It calculates a path from the start state to the goal state and returns the actions required to get from the start to the goal.

* **world** is the starting state representation for a navigation problem.
* **start** is the starting location, `(x, y)`.
* **goal** is the desired end position, `(x, y)`.
* **costs** is a `Dict` of costs for each type of terrain.
* **moves** is the legal movement model expressed in offsets.
* **heuristic** is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model.


The function returns the offsets needed to get from start state to the goal as a `List`. For example, for the test world:

```
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],

```

it would return:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

Do not make unwarranted assumptions. For example, do not assume the starting point is always `(0, 0)` or that the goal is always in the lower right hand corner.

In [25]:
def a_star_search( world, start, goal, costs, moves, heuristic ):
    frontier = {}
    exploredList = []

    # place initial state on frontier
    g_node = 0
    h_node = my_heuristic( start, goal )
    f_node = g_node + h_node
    
    # initialize frontier dictionary
    this_dict = {}
    this_dict["state"] = start
    this_dict["g"] = g_node
    this_dict["h"] = h_node
    this_dict["path"] = [start]
    key_as_tuple = (f_node, start)

    frontier[key_as_tuple] = this_dict

    while frontier:
        # determine which state has the lowest f(n) value on the frontier
        frontier_fn_entries = []
    
        # isolate f(n) values from tuple and determine lowest f(n)
        for dict_entry in frontier:
            this_fn = dict_entry[0]
            frontier_fn_entries.append( this_fn )
        lowest_fn = min( frontier_fn_entries )
    
        for dict_entry in frontier:
            this_fn = dict_entry[0]
            if this_fn == lowest_fn:
                next_state = dict_entry
                break
    
        # the state with the lowest f(n) becomes the current state
        currentStateDict = frontier[next_state]
        currentState = currentStateDict["state"]
    
        # pop the state with the lowest f(n) from the frontier
        del frontier[next_state]
    
        # check if the current state is the goal state
        if currentState == goal:
            path = ( currentStateDict["path"] )
        
        # obtain children / successors
        children = find_successors( currentStateDict, moves, world )
    
        # if there are no children, it is a terminal (but not necessarily goal) state
        if not children:
            continue
       
        for child_index in range( 0, len(children) ):
            # check if child is on explored list
            if children[child_index][0][1] in exploredList:
                continue        
        
            # check if child is on frontier already       
            duplicate_state_counter = 0
            for dict_index in frontier:
                child_state = children[child_index][0][1]
                frontier_state = dict_index[1]
                child_fn = children[child_index][0][0]
                frontier_fn = dict_index[0]
            
                if child_state == frontier_state:
                    duplicate_state_counter = duplicate_state_counter + 1
                    # if node is on frontier, check which has smaller f(n)
                    if child_fn < frontier_fn:
                        del frontier[dict_index]
                        frontier[children[child_index][0]] = children[child_index][1]
                        break                
        
            # if not on frontier or explored list, add to frontier        
            if duplicate_state_counter == 0:
                frontier[children[child_index][0]] = children[child_index][1]

        exploredList.append( currentState )
    return( path )

**pretty_print_solution**

The `pretty_print_solution` function prints an ASCII representation of the solution generated by the `a_star_search`. For example, for the test world, it would take the `world` and `path` and print:

```
v******
v******
v******
>>>>>>v
******v
******v
******G
```

using `v`, `^`, `>`, `<` to represent actions and `G` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.


Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

In [32]:
def pretty_print_solution( world, path, start):
    
    goal_state = path[(len( path ) - 1)]
    
    for path_index in range( 0, len(path) ):
        # isolate x and y coordinates from tuple
        startXcoord = path[path_index][0]
        startYcoord = path[path_index][1]
        
        # if it is the last item in the list,
        # it is the goal state
        if path[path_index] == goal_state:
            world[startYcoord][startXcoord] = "G"
            break
        
        # isolate x and y coordinates of the next state
        nextXcoord = path[path_index + 1][0]
        nextYcoord = path[path_index + 1][1]
        
        # calculate difference between x and y coordinates
        # of current state and the next state
        changeInX = nextXcoord - startXcoord
        changeInY = nextYcoord - startYcoord
        
        # determine whether the next state is up, down,
        # left, or right of the current state
            
        if changeInX == 1:
            world[startYcoord][startXcoord] = ">"
        
        if changeInX == -1:
            world[startYcoord][startXcoord] = "<"
            
        if changeInY == 1:
            world[startYcoord][startXcoord] = "v"
        
        if changeInY == -1:
            world[startYcoord][startXcoord] = "^"
    
    # change all other symbols to *
    direction_markers = [">", "<", "v", "^", "G"]
    
    for list_index in range( 0, len(world) ):
        for element_index in range( 0, len(world[list_index]) ):
            if world[list_index][element_index] not in direction_markers:
                world[list_index][element_index] = "*"
                
    for j in range( 0, len(world) ):
        print( "".join( world[j] ) )   
    
    return None

Execute `a_star_search` and `print_path` for the `test_world` and the `real_world`.

In [33]:
path = a_star_search(test_world, (0, 0), (len(test_world[0]) - 1, len(test_world) - 1), COSTS, MOVES, my_heuristic)
print(path)
pretty_print_solution(test_world, path, (0, 0))

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 4), (6, 5), (6, 6)]
v******
v******
v******
>>>>>>v
******v
******v
******G


In [34]:
path = a_star_search(full_world, (0, 0), (len(full_world[0]) - 1, len(full_world) - 1), COSTS, MOVES, my_heuristic)
print(path)
pretty_print_solution(full_world, path, (0, 0))

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (2, 21), (3, 21), (3, 22), (4, 22), (5, 22), (6, 22), (6, 23), (6, 24), (7, 24), (8, 24), (9, 24), (9, 25), (9, 26), (10, 26), (11, 26), (12, 26), (13, 26), (14, 26), (15, 26), (16, 26), (17, 26), (18, 26), (19, 26), (20, 26), (21, 26), (22, 26), (23, 26), (24, 26), (25, 26), (26, 26)]
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
v**************************
>v*************************
*v*************************
*v*************************
*v*************************
*v*********

## Think About...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.